## Loading from pickle, unordered concatenate problem

In [4]:
!conda install -y -c conda-forge s3fs

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda:

The following NEW packages will be INSTALLED:

    s3fs: 0.1.4-py_0 conda-forge

s3fs-0.1.4-py_ 100% |################################| Time: 0:00:00 222.52 kB/s


In [104]:
import pickle
import bisect

import iris
import s3fs
import numpy as np
from toolz.curried import *

## Loading pickled files from S3
One model run per file

In [29]:
fs = s3fs.S3FileSystem(anon=True)
files = fs.ls('mogreps-pickles/wet_bulb_potential_temperature')
files[:2]

['mogreps-pickles/wet_bulb_potential_temperature/mogreps-g_20161201_00_wet_bulb_potential_temperature.pickle',
 'mogreps-pickles/wet_bulb_potential_temperature/mogreps-g_20161202_00_wet_bulb_potential_temperature.pickle']

In [48]:
def unpickle(fname, prefix='/s3/'):
    with open('{}{}'.format(prefix, fname), 'rb') as inf:
        return pickle.load(inf)

cubes = iris.cube.CubeList(
    pipe(files, mapcat(unpickle)))
print(cubes)

0: wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 56; realization: 12; pressure: 3; latitude: 600; longitude: 800)
1: wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 56; realization: 12; pressure: 3; latitude: 600; longitude: 800)
2: wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 56; realization: 12; pressure: 3; latitude: 600; longitude: 800)
3: wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 56; realization: 12; pressure: 3; latitude: 600; longitude: 800)
4: wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 56; realization: 12; pressure: 3; latitude: 600; longitude: 800)
5: wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 56; realization: 12; pressure: 3; latitude: 600; longitude: 800)
6: wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 56; real

In [49]:
cubes = cubes.concatenate()
print(cubes)

0: wet_bulb_potential_temperature / (K) (forecast_reference_time: 29; forecast_period: 56; realization: 12; pressure: 3; latitude: 600; longitude: 800)
1: wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 56; realization: 11; pressure: 3; latitude: 600; longitude: 800)
2: wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 55; realization: 1; pressure: 3; latitude: 600; longitude: 800)
3: wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 1; realization: 1; pressure: 3; latitude: 600; longitude: 800)
4: wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 56; realization: 11; pressure: 3; latitude: 600; longitude: 800)
5: wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 55; realization: 1; pressure: 3; latitude: 600; longitude: 800)
6: wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 1; realizat

## Multistep concatenation
Why have these cubes not concatenated cleanly?

In [50]:
for c in cubes:
    print(c.coord('forecast_reference_time'))

DimCoord([2016-12-01 00:00:00, 2016-12-02 00:00:00, 2016-12-03 00:00:00,
       2016-12-04 00:00:00, 2016-12-05 00:00:00, 2016-12-06 00:00:00,
       2016-12-07 00:00:00, 2016-12-09 00:00:00, 2016-12-10 00:00:00,
       2016-12-11 00:00:00, 2016-12-12 00:00:00, 2016-12-13 00:00:00,
       2016-12-14 00:00:00, 2016-12-15 00:00:00, 2016-12-16 00:00:00,
       2016-12-17 00:00:00, 2016-12-18 00:00:00, 2016-12-19 00:00:00,
       2016-12-21 00:00:00, 2016-12-22 00:00:00, 2016-12-23 00:00:00,
       2016-12-24 00:00:00, 2016-12-25 00:00:00, 2016-12-26 00:00:00,
       2016-12-27 00:00:00, 2016-12-28 00:00:00, 2016-12-29 00:00:00,
       2016-12-30 00:00:00, 2016-12-31 00:00:00], standard_name='forecast_reference_time', calendar='gregorian')
DimCoord([2016-12-08 00:00:00], standard_name='forecast_reference_time', calendar='gregorian')
DimCoord([2016-12-08 00:00:00], standard_name='forecast_reference_time', calendar='gregorian')
DimCoord([2016-12-08 00:00:00], standard_name='forecast_referenc

The model runs for December 8th and 20th have not concatenated cleanly

In [87]:
frt_coord_points = lambda cube: cube.coord('forecast_reference_time').points
frt_filter = lambda cube: len(frt_coord_points(cube)) == 1
frt_grouper = lambda cube: frt_coord_points(cube)[0]

samples = pipe(
    cubes,
    filter(frt_filter),
    groupby(frt_grouper), valmap(iris.cube.CubeList))

In [90]:
print(samples)

{411432.0: [<iris 'Cube' of wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 56; realization: 11; pressure: 3; latitude: 600; longitude: 800)>,
<iris 'Cube' of wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 55; realization: 1; pressure: 3; latitude: 600; longitude: 800)>,
<iris 'Cube' of wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 1; realization: 1; pressure: 3; latitude: 600; longitude: 800)>], 411720.0: [<iris 'Cube' of wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 56; realization: 11; pressure: 3; latitude: 600; longitude: 800)>,
<iris 'Cube' of wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 55; realization: 1; pressure: 3; latitude: 600; longitude: 800)>,
<iris 'Cube' of wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 1; realization: 1; pressure: 3; latitude: 600; long

In [96]:
run1 = samples[411432.0]
print(run1)

0: wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 56; realization: 11; pressure: 3; latitude: 600; longitude: 800)
1: wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 55; realization: 1; pressure: 3; latitude: 600; longitude: 800)
2: wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 1; realization: 1; pressure: 3; latitude: 600; longitude: 800)


In [95]:
run1.concatenate_cube()

ConcatenateError: failed to concatenate into a single cube.
  An unexpected problem prevented concatenation.
  Expected only a single cube, found 3.

Visually it looks like there are two problems - one of the realisations is missing from the biggest cube, and it's missing because one of the forecast periods is missing from the second biggest cube.

In [101]:
for c in run1:
    print(c.coord('realization').points)
    
for c in run1:
    print(c.coord('forecast_period').points)

[ 0  1  2  4  5  6  7  8  9 10 11]
[3]
[3]
[   9.   12.   15.   18.   21.   24.   27.   30.   33.   36.   39.   42.
   45.   48.   51.   54.   57.   60.   63.   66.   69.   72.   75.   78.
   81.   84.   87.   90.   93.   96.   99.  102.  105.  108.  111.  114.
  117.  120.  123.  126.  129.  132.  135.  138.  141.  144.  147.  150.
  153.  156.  159.  162.  165.  168.  171.  174.]
[   9.   12.   15.   18.   21.   24.   27.   30.   33.   36.   39.   42.
   45.   48.   51.   54.   57.   60.   63.   66.   69.   72.   75.   78.
   81.   84.   87.   90.   93.   96.   99.  102.  105.  108.  111.  114.
  117.  120.  126.  129.  132.  135.  138.  141.  144.  147.  150.  153.
  156.  159.  162.  165.  168.  171.  174.]
[ 123.]


Step 1: Combine the two [3] realisation cubes

In [110]:
insert_index = bisect.bisect(
    run1[1].coord('forecast_period').points,
    run1[2].coord('forecast_period').points)
print(insert_index)

38


In [117]:
step1 = iris.cube.CubeList([
    run1[1][:, :insert_index],
    run1[2],
    run1[1][:, insert_index:]]).concatenate_cube()
step1

<iris 'Cube' of wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 56; realization: 1; pressure: 3; latitude: 600; longitude: 800)>

In [118]:
insert_index2 = bisect.bisect(
    run1[0].coord('realization').points,
    step1.coord('realization').points)
print(insert_index2)

3


In [121]:
run1_cube = iris.cube.CubeList([
    run1[0][:, :, :insert_index2],
    step1,
    run1[0][:, :, insert_index2:]]).concatenate_cube()
run1_cube

<iris 'Cube' of wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 56; realization: 12; pressure: 3; latitude: 600; longitude: 800)>

Now repeat the above for the second model run (likely bisecting forecast_period and realization again), then repeat again for the original cubelist (where you'll need to bisect forecast_reference_time), and they should all combine.



How would that work programatically? Some kind of recursive partitioning of cubes. Difficult part is identifying the co-ordinate to partion by.

Can't figure that out immediately, just gone for the copy paste hack below.

In [123]:
run2 = samples[411720.0]
insert_index = bisect.bisect(
    run2[1].coord('forecast_period').points,
    run2[2].coord('forecast_period').points)
step1 = iris.cube.CubeList([
    run2[1][:, :insert_index],
    run2[2],
    run2[1][:, insert_index:]]).concatenate_cube()
step1
insert_index2 = bisect.bisect(
    run2[0].coord('realization').points,
    step1.coord('realization').points)
print(insert_index2)
run2_cube = iris.cube.CubeList([
    run2[0][:, :, :insert_index2],
    step1,
    run2[0][:, :, insert_index2:]]).concatenate_cube()
run2_cube

1


<iris 'Cube' of wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 56; realization: 12; pressure: 3; latitude: 600; longitude: 800)>

In [125]:
result1 = iris.cube.CubeList([cubes[0], run1_cube, run2_cube])
result1

[<iris 'Cube' of wet_bulb_potential_temperature / (K) (forecast_reference_time: 29; forecast_period: 56; realization: 12; pressure: 3; latitude: 600; longitude: 800)>,
<iris 'Cube' of wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 56; realization: 12; pressure: 3; latitude: 600; longitude: 800)>,
<iris 'Cube' of wet_bulb_potential_temperature / (K) (forecast_reference_time: 1; forecast_period: 56; realization: 12; pressure: 3; latitude: 600; longitude: 800)>]

In [126]:
insert_index1 = bisect.bisect(
    result1[0].coord('forecast_reference_time').points,
    result1[1].coord('forecast_reference_time').points)
insert_index2 = bisect.bisect(
    result1[0].coord('forecast_reference_time').points,
    result1[2].coord('forecast_reference_time').points)
print(insert_index1, insert_index2)

7 18


In [129]:
finale = iris.cube.CubeList([
    result1[0][:insert_index1],
    result1[1],
    result1[0][insert_index1:insert_index2],
    result1[2],
    result1[0][insert_index2:]
]).concatenate_cube()

In [130]:
finale

<iris 'Cube' of wet_bulb_potential_temperature / (K) (forecast_reference_time: 31; forecast_period: 56; realization: 12; pressure: 3; latitude: 600; longitude: 800)>

In [ ]:
with open('./mogreps-g_201612_00_wet_bulb_potential_temperature.pickle', 'wb') as outf:
    pickle.dump(finale,)